Import libraries

In [1]:
#to pre-process data
import pandas as pd
from pandas import DataFrame
#to save into json and csv file formats
import json
from pandas.io.json import json_normalize
#to process mesh terms
import re

Import mesh terms

In [2]:
terms, ui_terms, ui_numbers, number_uis = {}, {}, {}, {}
numbers = []
 
meshFile = 'd2020.bin'
with open(meshFile, mode='rb') as file:
    mesh = file.readlines()


Parse mesh terms, retrieve mesh terms, mesh IDs

In [3]:
for line in mesh:
    #print(line)
    meshTerm = re.search(b'MH = (.+)$', line)
    if meshTerm:
        term = meshTerm.group(1).decode('utf-8')
        
    meshNumber = re.search(b'MN = (.+)$', line)
    if meshNumber:
        numbers.append(meshNumber.group(1).decode('utf-8'))
        
    meshUI = re.search(b'UI = (.+)$', line)
    if meshUI:
        ui = meshUI.group(1).decode('utf-8')
        ui_terms[ui] = term
        ui_numbers[ui] = numbers
        
        if term in terms:
            terms[term] = terms[term] + ' ' + ui
        else:
            terms[term] = ui
        for numb in numbers:
            if numb in number_uis:
                number_uis[numb] = number_uis[numb] + ' ' + ui
            else:
                number_uis[numb] = ui
        numbers = []

Test that parsing and retrieval works

In [4]:
for line in mesh[:200]:
    print(line)
    meshTerm = re.search(b'MH = (.+)$', line)
    if meshTerm:
        term = meshTerm.group(1).decode('utf-8')
        
    meshNumber = re.search(b'MN = (.+)$', line)
    if meshNumber:
        numbers.append(meshNumber.group(1).decode('utf-8'))
        
    meshUI = re.search(b'UI = (.+)$', line)
    if meshUI:
        ui = meshUI.group(1).decode('utf-8')
        ui_terms[ui] = term
        ui_numbers[ui] = numbers
        
        if term in terms:
            terms[term] = terms[term] + ' ' + ui
        else:
            terms[term] = ui
        for numb in numbers:
            if numb in number_uis:
                number_uis[numb] = number_uis[numb] + ' ' + ui
            else:
                number_uis[numb] = ui
        numbers = []

b'*NEWRECORD\n'
b'RECTYPE = D\n'
b'MH = Calcimycin\n'
b'AQ = AA AD AE AG AI AN BI BL CF CH CL CS EC HI IM IP ME PD PK PO RE SD ST TO TU UR\n'
b'ENTRY = A-23187|T109|T195|LAB|NRW|NLM (1991)|900308|abbcdef\n'
b'ENTRY = A23187|T109|T195|LAB|NRW|UNK (19XX)|741111|abbcdef\n'
b'ENTRY = Antibiotic A23187|T109|T195|NON|NRW|NLM (1991)|900308|abbcdef\n'
b'ENTRY = A 23187\n'
b'ENTRY = A23187, Antibiotic\n'
b'MN = D03.633.100.221.173\n'
b'PA = Anti-Bacterial Agents\n'
b'PA = Calcium Ionophores\n'
b'MH_TH = FDA SRS (2014)\n'
b'MH_TH = NLM (1975)\n'
b'ST = T109\n'
b'ST = T195\n'
b'N1 = 4-Benzoxazolecarboxylic acid, 5-(methylamino)-2-((3,9,11-trimethyl-8-(1-methyl-2-oxo-2-(1H-pyrrol-2-yl)ethyl)-1,7-dioxaspiro(5.5)undec-2-yl)methyl)-, (6S-(6alpha(2S*,3S*),8beta(R*),9beta,11alpha))-\n'
b'RN = 37H9VM9WZL\n'
b'RR = 52665-69-7 (Calcimycin)\n'
b'PI = Antibiotics (1973-1974)\n'
b'PI = Carboxylic Acids (1973-1974)\n'
b'MS = An ionophorous, polyether antibiotic from Streptomyces chartreusensis. It binds and t

Test on the data

In [5]:
print(ui_numbers['D015470'][0][:3])
print(ui_terms[number_uis[ui_numbers['D015470'][0][:]]])
print(number_uis['D06.472'])
print(ui_terms[number_uis['J01']])
print(ui_terms['D000906'])

C04
Leukemia, Myeloid, Acute
D006728
Technology, Industry, and Agriculture
Antibodies


Create a first level disease class

In [6]:
get_disease = lambda x : ui_numbers[x][0][:3]+' '+ui_terms[number_uis[ui_numbers[x][0][:3]]]

Csv file to process. The file includes all approved oral compounds that are not discontinued as products and are not withdrawn from the market and what have been classified with a disease class (mesh_id). Essentially all these compounds come from ChEMBL v26.

In [7]:
df_very_small_companies_recent_oral_compounds=pd.read_csv('./DATA/very_small_companies_last_2_years.csv')

In [8]:
df_very_small_companies_recent_oral_compounds.head()

,molregno,mesh_id,pref_name,company,mw,clogp,arom,hba,hbd,rtb,psa,bpka,apka
0,2261,D001007,CLOZAPINE,HERITAGE LIFE SCIENCES BARBADOS INC,326.83,3.4,2.0,4.0,1.0,0.0,30.87,7.35,NaN
1,2261,D001714,CLOZAPINE,HERITAGE LIFE SCIENCES BARBADOS INC,326.83,3.4,2.0,4.0,1.0,0.0,30.87,7.35,NaN
2,2261,D003704,CLOZAPINE,HERITAGE LIFE SCIENCES BARBADOS INC,326.83,3.4,2.0,4.0,1.0,0.0,30.87,7.35,NaN
3,2261,D003866,CLOZAPINE,HERITAGE LIFE SCIENCES BARBADOS INC,326.83,3.4,2.0,4.0,1.0,0.0,30.87,7.35,NaN
4,2261,D011618,CLOZAPINE,HERITAGE LIFE SCIENCES BARBADOS INC,326.83,3.4,2.0,4.0,1.0,0.0,30.87,7.35,NaN


In [9]:
len(df_very_small_companies_recent_oral_compounds)

681

In [10]:
df_larger_companies = pd.read_csv('./DATA/larger_companies.csv')

In [11]:
df_larger_companies.head()

,molregno,mesh_id,pref_name,company,mw,clogp,arom,hba,hbd,rtb,psa,bpka,apka
0,115,D000375,NICOTINE,PHARMACIA AND UPJOHN CO,162.24,1.16,1.0,2.0,0.0,1.0,16.13,8.58,NaN
1,115,D000428,NICOTINE,PHARMACIA AND UPJOHN CO,162.24,1.16,1.0,2.0,0.0,1.0,16.13,8.58,NaN
2,115,D001289,NICOTINE,PHARMACIA AND UPJOHN CO,162.24,1.16,1.0,2.0,0.0,1.0,16.13,8.58,NaN
3,115,D002189,NICOTINE,PHARMACIA AND UPJOHN CO,162.24,1.16,1.0,2.0,0.0,1.0,16.13,8.58,NaN
4,115,D002277,NICOTINE,PHARMACIA AND UPJOHN CO,162.24,1.16,1.0,2.0,0.0,1.0,16.13,8.58,NaN


In [12]:
len(df_larger_companies)

133449

Merge the file with no data on very small companies compounds (df_larger_companies) and a file with all the data on very small companies compounds that have been approved since Jan-01-2018 (df_very_small_companies_recent_oral_compounds).

In [13]:

#df_oral_compounds=pd.concat([df_very_small_companies_recent_oral_compounds, df_larger_companies]).drop_duplicates().reset_index(drop=True)
df_oral_compounds=pd.concat([df_very_small_companies_recent_oral_compounds, df_larger_companies])
len(df_oral_compounds)

134130

A list of very small companies.

In [14]:
very_small_companies= ['ABHAI INC',
 'ACORDA THERAPEUTICS INC',
 'ACTAVIS INC',
 'ACTAVIS TOTOWA LLC AN INDIRECT WHOLLY OWNED SUB OF TEVA PHARMACEUTICALS USA INC',
 'ADAMAS PHARMA LLC',
 'ADARE PHARMACEUTICALS INC',
 'ADDMEDICA SAS',
 'AGOURON PHARMACEUTICALS LLC',
 'AIZANT DRUG RESEARCH SOLUTIONS PRIVATE LTD',
 'ALKALOIDA CHEMICAL CO ZRT',
 'ALLEGIS HOLDINGS LLC',
 'ALVOGEN GROUP HOLDINGS 4 LLC',
 'ALVOGEN PINE BROOK LLC',
 'AMARIN PHARMACEUTICALS IRELAND LTD',
 'AMGEN INC',
 'AMICI PHARMACEUTICALS LLC',
 'AMNEAL PHARMACEUTICALS HOLDINGS GMBH',
 'ANBEX INC',
 'ANCHEN PHARMACEUTICALS TAIWAN INC',
 'ANDA REPOSITORY LLC',
 'ANDOR PHARMACEUTICALS LLC',
 'ANTRIM PHARMACEUTICALS LLC',
 'APEX PHARMACEUTICALS INC',
 'APICORE US LLC',
 'APOTEX TECHNOLOGIES INC',
 'AQUESTIVE THERAPEUTICS',
 'ARCO PHARMACEUTICALS LLC',
 'ASTELLAS PHARMA GLOBAL DEVELOPMENT INC',
 'ASTRAZENECA LP',
 'ASTRAZENECA UK LTD',
 'ATHENA BIOSCIENCES LLC',
 'AUCTA PHARMACEUTICALS INC',
 'AYTU BIOSCIENCE INC',
 'BIOCON LTD',
 'BRISTOL MYERS SQUIBB CO PHARMACEUTICAL RESEARCH INSTITUTE',
 'BRISTOL MYERS SQUIBB PHARMA CO',
 'CAPELLON PHARMACEUTICALS LLC',
 'CERECOR INC',
 'CHANGZHOU PHARMACEUTICAL FACTORY',
 'CHATTEM INC',
 'CHEMI SPA',
 'CHEPLAPHARM ARZNEIMITTEL GMBH',
 'CHINA RESOURCES SAIKE PHARMACEUTICAL CO LTD',
 'CLOVER PHARMACEUTICALS CORP',
 'CLOVIS ONCOLOGY INC',
 'COEPTIS PHARMACEUTICALS INC',
 'CORDEN PHARMA LATINA SPA',
 'CP PHARMACEUTICALS INTERNATIONAL CV',
 'CROSSMEDIKA SA',
 'CUBIST PHARMACEUTICALS INC',
 'CUBIST PHARMACEUTICALS LLC',
 'DAVA INTERNATIONAL INC',
 'DBL PHARMACEUTICALS INC',
 'DEPO NF SUB LLC A SUB OF ASSERTIO THERAPEUTICS INC',
 'DEVA HOLDING AS',
 'DOW PHARMACEUTICAL SCIENCES',
 'EPI HEALTH LLC',
 'EXALENZ BIOSCIENCE LTD',
 'EXELA PHARMA SCIENCES LLC',
 'FOREST LABORATORIES LLC',
 'FRESENIUS KABI AUSTRIA GMBH',
 'FRESENIUS MEDICAL CARE NORTH AMERICA',
 'G AND W LABORATORIES INC',
 'GATE PHARMACEUTICALS',
 'GE HEALTHCARE',
 'GENENTECH INC',
 'GENZYME CORP',
 'GLENMARK PHARMACEUTICALS INC',
 'GRANULES PHARMACEUTICALS INC',
 'HELSINN HEALTHCARE SA',
 'HERITAGE LIFE SCIENCES BARBADOS INC',
 'HOFFMANN-LA ROCHE INC',
 'HORIZON THERAPEUTICS LLC',
 'INCYTE CORP',
 'INSTITUT BIOCHIMIQUE SA (IBSA)',
 'INTERPHARMA PRAHA AS',
 'IPR PHARMACEUTICALS INC',
 'IRONSHORE PHARMACEUTICALS AND DEVELOPMENT INC',
 'ITALFARMACO SPA',
 'JANSSEN BIOTECH INC',
 'JANSSEN RESEARCH AND DEVELOPMENT LLC',
 'JAZZ PHARMACEUTICALS INC',
 'JAZZ PHARMACEUTICALS IRELAND LTD',
 'JIANGXI BOYA SEEHOT PHARMACEUTICAL CO LTD',
 'JUBILANT DRAXIMAGE INC',
 'KADMON PHARMACEUTICALS LLC',
 'KOWA CO LTD',
 'KYOWA KIRIN INC',
 'LABORATORIE HRA PHARMA',
 'LANDELA PHARMACEUTICAL',
 'LAX PHARMA LLC',
 'LG CHEM LTD',
 'LIFEPHARMA FZE',
 'MALLINCKRODT INC',
 'MANKIND PHARMA LTD',
 'MCNEIL CONSUMER HEALTHCARE',
 'MEDICIS PHARMACEUTICAL CORP',
 'MERZ PHARMACEUTICALS LLC',
 'METUCHEN PHARMACEUTICALS LLC',
 'MILLENNIUM PHARMACEUTICALS INC',
 'MYLAN PHARMACEUTICALS INC.',
 'NATCO PHARMA LIMITED',
 'NEOS THERAPEUTICS',
 'NEXTWAVE PHARMACEUTICALS INC',
 'NEXTWAVE PHARMACEUTICALS INC A SUB OF TRIS PHARMA INC',
 'NOVA LABORATORIES LTD',
 'NOVELGENIX THERAPEUTICS PVT LTD',
 'NX DEVELOPMENT CORP',
 'ODYSSEY PHARMACEUTICALS INC',
 'OPKO IRELAND GLOBAL HOLDINGS LTD',
 'OSI PHARMACEUTICALS LLC',
 'OSMOTICA PHARMACEUTICAL',
 'OUTLOOK PHARMACEUTICALS INC',
 'PANACEA BIOTEC LTD',
 'PAR STERILE PRODUCTS LLC',
 'PARKE-DAVIS DIVISION OF PFIZER INC',
 'PD PARTNERS IV LLC',
 'PFIZER CONSUMER HEALTHCARE',
 'PFIZER PHARMACEUTICALS PRODUCTION CORP LTD',
 'PHARMACIA AND UPJOHN',
 'PLIVA PHARMACEUTICAL INDUSTRY INC',
 'PRAGMA PHARMACEUTICALS LLC',
 'PROVELL PHARMACEUTICALS LLC',
 'PUMA BIOTECHNOLOGY INC',
 'RECIP AB',
 'RECIPHARM PHARMASERVICES PRIVATE LTD',
 'RECRO GAINESVILLE LLC',
 'ROMARK LABORATORIES',
 'RXMTM THERAPEUTICS LLC A WHOLLY OWNED SUB OF CUTISPHARMA INC',
 'SAGENT PHARMACEUTICALS INC',
 'SANOFI GENZYME',
 'SANOFI US',
 'SAWAI USA INC',
 'SCHERING CORP',
 'SECURA BIO INC',
 'SETON PHARMACEUTICAL LLC',
 'SHANDONG NEW TIME PHARMACEUTICAL CO LTD',
 'SHIONOGI INC',
 'SIDMAK LABORATORIES INDIA PVT LTD',
 'SOFGEN PHARMACEUTICALS',
 'SQUARE PHARMACEUTICALS LTD',
 'STASON PHARMACEUTICALS INC',
 'STIEFEL LABORATORIES INC',
 'STRIDES VIVIMED PTE LTD',
 'STRONGBRIDGE US INC',
 'SUCAMPO PHARMA AMERICAS LLC',
 'SUN PHARMA GLOBAL INC',
 'SUNNY PHARMTECH INC',
 'SWEDISH ORPHAN BIOVITRUM AB PUBL',
 'TAIHO ONCOLOGY INC',
 'TASMAN PHARMA INC',
 'TESARO INC',
 'TEVA NEUROSCIENCE INC',
 'TEVA PHARMACEUTICALS INTERNATIONAL GMBH',
 'TIME-CAP LABORATORIES INC',
 'TOPROL ACQUISITION LLC',
 'TORPHARM INC',
 'TORRENT PHARMACEUTICALS LTD.',
 'TULEX PHARMACEUTICALS INC',
 'UNICHEM LABORATORIES LIMITED',
 'UNICHEM PHARMACEUTICALS (USA) INC',
 'UNIMARK REMEDIES LTD',
 'UPSHER SMITH LABORATORIES INC',
 'US WORLDMEDS LLC',
 'USPHARMA WINDLAS LLC',
 'VALEANT INTERNATIONAL BARBADOS SRL',
 'VALEANT PHARMACEUTICALS INTERNATIONAL INC',
 'VALEANT PHARMACEUTICALS LUXEMBOURG SARL',
 'VALIDUS PHARMACEUTICALS INC',
 'VANDA PHARMACEUTICALS INC',
 'VELOXIS PHARMACEUTICALS INC',
 'VEROSCIENCE LLC',
 'VERTEX PHARMACEUTICALS INC',
 'VKT PHARMA PRIVATE LTD',
 'VYERA PHARMACEUTICALS LLC',
 'WINDLAS HEALTHCARE PVT LTD',
 'XIAMEN LP PHARMACUETICAL CO LTD',
 'YABAO PHARMACEUTICAL CO LTD BEIJING',
 'ZAMBON SPA ITALY']

Generate a disease class from mesh_id of the compound. The disease classes come from Mesh Terms.

In [15]:
df_oral_compounds['disease'] = df_oral_compounds['mesh_id'].apply(get_disease)

Fill all empty cells (NaN) with -1000

In [16]:
df_oral_compounds.fillna(-1000)

,molregno,mesh_id,pref_name,company,mw,clogp,arom,hba,hbd,rtb,psa,bpka,apka,disease
0,2261,D001007,CLOZAPINE,HERITAGE LIFE SCIENCES BARBADOS INC,326.83,3.40,2.0,4.0,1.0,0.0,30.87,7.35,-1000.00,F01 Behavior and Behavior Mechanisms
1,2261,D001714,CLOZAPINE,HERITAGE LIFE SCIENCES BARBADOS INC,326.83,3.40,2.0,4.0,1.0,0.0,30.87,7.35,-1000.00,F03 Mental Disorders
2,2261,D003704,CLOZAPINE,HERITAGE LIFE SCIENCES BARBADOS INC,326.83,3.40,2.0,4.0,1.0,0.0,30.87,7.35,-1000.00,C10 Nervous System Diseases
3,2261,D003866,CLOZAPINE,HERITAGE LIFE SCIENCES BARBADOS INC,326.83,3.40,2.0,4.0,1.0,0.0,30.87,7.35,-1000.00,F03 Mental Disorders
4,2261,D011618,CLOZAPINE,HERITAGE LIFE SCIENCES BARBADOS INC,326.83,3.40,2.0,4.0,1.0,0.0,30.87,7.35,-1000.00,F03 Mental Disorders
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133444,2335118,D011618,HALOPERIDOL LACTATE,LANNETT CO INC,375.87,3.66,2.0,3.0,1.0,6.0,40.54,8.05,13.96,F03 Mental Disorders
133445,2335118,D011618,HALOPERIDOL LACTATE,PHARMACEUTICAL ASSOCIATES INC,375.87,3.66,2.0,3.0,1.0,6.0,40.54,8.05,13.96,F03 Mental Disorders
133446,2335118,D012559,HALOPERIDOL LACTATE,LANNETT CO INC,375.87,3.66,2.0,3.0,1.0,6.0,40.54,8.05,13.96,F03 Mental Disorders
133447,2335118,D012559,HALOPERIDOL LACTATE,PHARMACEUTICAL ASSOCIATES INC,375.87,3.66,2.0,3.0,1.0,6.0,40.54,8.05,13.96,F03 Mental Disorders


Create two separate dataframes - one for company->disease class->counts and the other one for company->disease class-> compound name-> descriptors (mw, clogp, arom, hba, hbd, rtb, psa, bpka, apka)

In [17]:
df_counts= df_oral_compounds[['company', 'disease', 'pref_name']].copy()
df_descriptors_companies=df_oral_compounds[['company', 'pref_name', 'mw', 'clogp', 'arom', 'hba', 'hbd','rtb', 'psa', 'bpka', 'apka']].copy()
df_descriptors_diseases=df_oral_compounds[['company','disease', 'pref_name', 'mw', 'clogp', 'arom', 'hba', 'hbd','rtb', 'psa', 'bpka', 'apka']].copy()

Group by preffered name

In [18]:
df_counts.groupby(['company', 'disease', 'pref_name']).count().head(5)

Empty DataFrame
Columns: []
Index: [(AAIPHARMA LLC, C04 Neoplasms, AZATHIOPRINE), (AAIPHARMA LLC, C05 Musculoskeletal Diseases, AZATHIOPRINE), (AAIPHARMA LLC, C06 Digestive System Diseases, AZATHIOPRINE), (AAIPHARMA LLC, C08 Respiratory Tract Diseases, AZATHIOPRINE), (AAIPHARMA LLC, C10 Nervous System Diseases, AZATHIOPRINE)]

In [19]:
df_counts.columns =['company', 'disease_class', 'compound_name']

In [20]:
df_counts.columns

Index(['company', 'disease_class', 'compound_name'], dtype='object')

In [21]:
ddf_counts = df_counts.groupby(['company', 'disease_class'])['compound_name'].value_counts()

In [22]:
ddf_counts.head()

company        disease_class                   compound_name
AAIPHARMA LLC  C04 Neoplasms                   AZATHIOPRINE      4
               C05 Musculoskeletal Diseases    AZATHIOPRINE      4
               C06 Digestive System Diseases   AZATHIOPRINE      8
               C08 Respiratory Tract Diseases  AZATHIOPRINE     10
               C10 Nervous System Diseases     AZATHIOPRINE      4
Name: compound_name, dtype: int64

In [23]:
ddf_counts_df = pd.DataFrame(ddf_counts)

In [24]:
ddf_counts_df.columns =['counts']

In [25]:
ddf_counts_df.drop(columns='counts')

Empty DataFrame
Columns: []
Index: [(AAIPHARMA LLC, C04 Neoplasms, AZATHIOPRINE), (AAIPHARMA LLC, C05 Musculoskeletal Diseases, AZATHIOPRINE), (AAIPHARMA LLC, C06 Digestive System Diseases, AZATHIOPRINE), (AAIPHARMA LLC, C08 Respiratory Tract Diseases, AZATHIOPRINE), (AAIPHARMA LLC, C10 Nervous System Diseases, AZATHIOPRINE), (AAIPHARMA LLC, C12 Male Urogenital Diseases, AZATHIOPRINE), (AAIPHARMA LLC, C14 Cardiovascular Diseases, AZATHIOPRINE), (AAIPHARMA LLC, C15 Hemic and Lymphatic Diseases, AZATHIOPRINE), (AAIPHARMA LLC, C16 Congenital, Hereditary, and Neonatal Diseases and Abnormalities, AZATHIOPRINE), (AAIPHARMA LLC, C17 Skin and Connective Tissue Diseases, AZATHIOPRINE), (AAIPHARMA LLC, C20 Immune System Diseases, AZATHIOPRINE), (ABBVIE ENDOCRINE INC, C04 Neoplasms, LEUPROLIDE ACETATE), (ABBVIE ENDOCRINE INC, C04 Neoplasms, NORETHINDRONE ACETATE), (ABBVIE ENDOCRINE INC, C05 Musculoskeletal Diseases, NORETHINDRONE ACETATE), (ABBVIE ENDOCRINE INC, C10 Nervous System Diseases, LEUPROLIDE ACETATE), (ABBVIE ENDOCRINE INC, C12 Male Urogenital Diseases, LEUPROLIDE ACETATE), (ABBVIE ENDOCRINE INC, C13 Female Urogenital Diseases and Pregnancy Complications, LEUPROLIDE ACETATE), (ABBVIE ENDOCRINE INC, C13 Female Urogenital Diseases and Pregnancy Complications, NORETHINDRONE ACETATE), (ABBVIE ENDOCRINE INC, C14 Cardiovascular Diseases, LEUPROLIDE ACETATE), (ABBVIE ENDOCRINE INC, C17 Skin and Connective Tissue Diseases, NORETHINDRONE ACETATE), (ABBVIE ENDOCRINE INC, C18 Nutritional and Metabolic Diseases, LEUPROLIDE ACETATE), (ABBVIE ENDOCRINE INC, C19 Endocrine System Diseases, LEUPROLIDE ACETATE), (ABBVIE ENDOCRINE INC, C19 Endocrine System Diseases, NORETHINDRONE ACETATE), (ABBVIE ENDOCRINE INC, C23 Pathological Conditions, Signs and Symptoms, NORETHINDRONE ACETATE), (ABBVIE ENDOCRINE INC, G08 Reproductive and Urinary Physiological Phenomena, NORETHINDRONE ACETATE), (ABBVIE INC, C01 Infections, RITONAVIR), (ABBVIE INC, C01 Infections, CYCLOSPORINE), (ABBVIE INC, C01 Infections, LOPINAVIR), (ABBVIE INC, C01 Infections, SIMVASTATIN), (ABBVIE INC, C01 Infections, NIACIN), (ABBVIE INC, C01 Infections, VALPROIC ACID), (ABBVIE INC, C01 Infections, DASABUVIR SODIUM), (ABBVIE INC, C01 Infections, LOVASTATIN), (ABBVIE INC, C01 Infections, DRONABINOL), (ABBVIE INC, C01 Infections, FENOFIBRATE), (ABBVIE INC, C04 Neoplasms, CYCLOSPORINE), (ABBVIE INC, C04 Neoplasms, SIMVASTATIN), (ABBVIE INC, C04 Neoplasms, RITONAVIR), (ABBVIE INC, C04 Neoplasms, VALPROIC ACID), (ABBVIE INC, C04 Neoplasms, LOVASTATIN), (ABBVIE INC, C04 Neoplasms, NIACIN), (ABBVIE INC, C04 Neoplasms, PARICALCITOL), (ABBVIE INC, C04 Neoplasms, DIVALPROEX SODIUM), (ABBVIE INC, C04 Neoplasms, FENOFIBRATE), (ABBVIE INC, C04 Neoplasms, LEVOTHYROXINE SODIUM), (ABBVIE INC, C05 Musculoskeletal Diseases, CYCLOSPORINE), (ABBVIE INC, C05 Musculoskeletal Diseases, SIMVASTATIN), (ABBVIE INC, C05 Musculoskeletal Diseases, LOVASTATIN), (ABBVIE INC, C05 Musculoskeletal Diseases, DRONABINOL), (ABBVIE INC, C05 Musculoskeletal Diseases, LEVOTHYROXINE SODIUM), (ABBVIE INC, C06 Digestive System Diseases, SIMVASTATIN), (ABBVIE INC, C06 Digestive System Diseases, RITONAVIR), (ABBVIE INC, C06 Digestive System Diseases, CYCLOSPORINE), (ABBVIE INC, C06 Digestive System Diseases, FENOFIBRATE), (ABBVIE INC, C06 Digestive System Diseases, DRONABINOL), (ABBVIE INC, C07 Stomatognathic Diseases, CYCLOSPORINE), (ABBVIE INC, C07 Stomatognathic Diseases, SIMVASTATIN), (ABBVIE INC, C08 Respiratory Tract Diseases, RITONAVIR), (ABBVIE INC, C08 Respiratory Tract Diseases, CYCLOSPORINE), (ABBVIE INC, C08 Respiratory Tract Diseases, SIMVASTATIN), (ABBVIE INC, C08 Respiratory Tract Diseases, LOVASTATIN), (ABBVIE INC, C08 Respiratory Tract Diseases, DRONABINOL), (ABBVIE INC, C10 Nervous System Diseases, DIVALPROEX SODIUM), (ABBVIE INC, C10 Nervous System Diseases, SIMVASTATIN), (ABBVIE INC, C10 Nervous System Diseases, LOVASTATIN), (ABBVIE INC, C10 Nervous System Diseases, VALPROIC ACID), (ABBVIE INC, C10 Nervous Syst

In [26]:
df_counts_to_json=ddf_counts_df.groupby(['company', 'disease_class']).count().reset_index(['disease_class'])

In [27]:
df_counts_to_json.head(100)

,disease_class,counts
company,,
AAIPHARMA LLC,C04 Neoplasms,1
AAIPHARMA LLC,C05 Musculoskeletal Diseases,1
AAIPHARMA LLC,C06 Digestive System Diseases,1
AAIPHARMA LLC,C08 Respiratory Tract Diseases,1
AAIPHARMA LLC,C10 Nervous System Diseases,1
...,...,...
ACELLA PHARMACEUTICALS LLC,C09 Otorhinolaryngologic Diseases,1
ACELLA PHARMACEUTICALS LLC,C10 Nervous System Diseases,4
ACELLA PHARMACEUTICALS LLC,C13 Female Urogenital Diseases and Pregnancy C...,1


Save in the json file

In [28]:
df_counts_to_json.to_json('./DATA/cdc_data.json', orient='table')

Test json file

In [29]:
with open('./DATA/cdc_data.json') as data_file:
        
    data_as_records=json.load(data_file)

In [30]:
data_as_records['data']

[{'company': 'AAIPHARMA LLC', 'disease_class': 'C04 Neoplasms', 'counts': 1},
 {'company': 'AAIPHARMA LLC',
  'disease_class': 'C05 Musculoskeletal Diseases',
  'counts': 1},
 {'company': 'AAIPHARMA LLC',
  'disease_class': 'C06 Digestive System Diseases',
  'counts': 1},
 {'company': 'AAIPHARMA LLC',
  'disease_class': 'C08 Respiratory Tract Diseases',
  'counts': 1},
 {'company': 'AAIPHARMA LLC',
  'disease_class': 'C10 Nervous System Diseases',
  'counts': 1},
 {'company': 'AAIPHARMA LLC',
  'disease_class': 'C12 Male Urogenital Diseases',
  'counts': 1},
 {'company': 'AAIPHARMA LLC',
  'disease_class': 'C14 Cardiovascular Diseases',
  'counts': 1},
 {'company': 'AAIPHARMA LLC',
  'disease_class': 'C15 Hemic and Lymphatic Diseases',
  'counts': 1},
 {'company': 'AAIPHARMA LLC',
  'disease_class': 'C16 Congenital, Hereditary, and Neonatal Diseases and Abnormalities',
  'counts': 1},
 {'company': 'AAIPHARMA LLC',
  'disease_class': 'C17 Skin and Connective Tissue Diseases',
  'counts'

In [31]:
ddf_descriptors_diseases=df_descriptors_diseases.drop_duplicates().reset_index()

In [32]:
ddf_descriptors_companies=df_descriptors_companies.drop_duplicates().reset_index()
ddf_descriptors_companies=ddf_descriptors_diseases.drop(columns='index')
ddf_descriptors_companies.rename(columns = {'disease':'disease_class', 'pref_name':'compound'}, inplace = True) 
ddf_descriptors_companies=ddf_descriptors_companies.sort_values(by='company')

In [33]:
len(ddf_descriptors_diseases)

26878

In [34]:
ddf_descriptors_diseases.groupby(['company', 'disease'])['pref_name']

In [35]:
ddf_descriptors_diseases.head()

,index,company,disease,pref_name,mw,clogp,arom,hba,hbd,rtb,psa,bpka,apka
0,0,HERITAGE LIFE SCIENCES BARBADOS INC,F01 Behavior and Behavior Mechanisms,CLOZAPINE,326.83,3.40,2.0,4.0,1.0,0.0,30.87,7.35,NaN
1,1,HERITAGE LIFE SCIENCES BARBADOS INC,F03 Mental Disorders,CLOZAPINE,326.83,3.40,2.0,4.0,1.0,0.0,30.87,7.35,NaN
2,2,HERITAGE LIFE SCIENCES BARBADOS INC,C10 Nervous System Diseases,CLOZAPINE,326.83,3.40,2.0,4.0,1.0,0.0,30.87,7.35,NaN
3,6,ALVOGEN GROUP HOLDINGS 2 LLC,C04 Neoplasms,ETHACRYNIC ACID,303.14,3.66,1.0,3.0,1.0,6.0,63.60,NaN,2.8
4,7,ALVOGEN GROUP HOLDINGS 2 LLC,C14 Cardiovascular Diseases,ETHACRYNIC ACID,303.14,3.66,1.0,3.0,1.0,6.0,63.60,NaN,2.8


In [36]:
len(ddf_descriptors_diseases)

26878

In [37]:
ddf_descriptors_diseases_no_old_index=ddf_descriptors_diseases.drop(columns='index')
ddf_descriptors_diseases_no_old_index.rename(columns = {'disease':'disease_class', 'pref_name':'compound'}, inplace = True)
ddf_descriptors_diseases_no_old_index=ddf_descriptors_diseases_no_old_index.sort_values(by='company')
ddf_descriptors_diseases_no_old_index.head()

,company,disease_class,compound,mw,clogp,arom,hba,hbd,rtb,psa,bpka,apka
17964,AAIPHARMA LLC,"C16 Congenital, Hereditary, and Neonatal Disea...",AZATHIOPRINE,277.27,1.17,3.0,8.0,1.0,3.0,115.42,2.16,8.62
17979,AAIPHARMA LLC,C15 Hemic and Lymphatic Diseases,AZATHIOPRINE,277.27,1.17,3.0,8.0,1.0,3.0,115.42,2.16,8.62
17974,AAIPHARMA LLC,C10 Nervous System Diseases,AZATHIOPRINE,277.27,1.17,3.0,8.0,1.0,3.0,115.42,2.16,8.62
17949,AAIPHARMA LLC,C04 Neoplasms,AZATHIOPRINE,277.27,1.17,3.0,8.0,1.0,3.0,115.42,2.16,8.62
17969,AAIPHARMA LLC,C14 Cardiovascular Diseases,AZATHIOPRINE,277.27,1.17,3.0,8.0,1.0,3.0,115.42,2.16,8.62


In [38]:
len(ddf_descriptors_companies)

26878

In [39]:
ddf_descriptors_companies.to_json('./DATA/cdc_descriptors_companies.json', orient='table')

In [40]:
ddf_descriptors_diseases_no_old_index.to_json('./DATA/cdc_descriptors_diseases.json', orient='table')

In [41]:
with open('./DATA/cdc_descriptors_companies.json') as data_file:
        
    descriptors_companies_as_records=json.load(data_file)

In [42]:
descriptors_companies_as_records['data']

[{'index': 17964,
  'company': 'AAIPHARMA LLC',
  'disease_class': 'C16 Congenital, Hereditary, and Neonatal Diseases and Abnormalities',
  'compound': 'AZATHIOPRINE',
  'mw': 277.27,
  'clogp': 1.17,
  'arom': 3.0,
  'hba': 8.0,
  'hbd': 1.0,
  'rtb': 3.0,
  'psa': 115.42,
  'bpka': 2.16,
  'apka': 8.62},
 {'index': 17979,
  'company': 'AAIPHARMA LLC',
  'disease_class': 'C15 Hemic and Lymphatic Diseases',
  'compound': 'AZATHIOPRINE',
  'mw': 277.27,
  'clogp': 1.17,
  'arom': 3.0,
  'hba': 8.0,
  'hbd': 1.0,
  'rtb': 3.0,
  'psa': 115.42,
  'bpka': 2.16,
  'apka': 8.62},
 {'index': 17974,
  'company': 'AAIPHARMA LLC',
  'disease_class': 'C10 Nervous System Diseases',
  'compound': 'AZATHIOPRINE',
  'mw': 277.27,
  'clogp': 1.17,
  'arom': 3.0,
  'hba': 8.0,
  'hbd': 1.0,
  'rtb': 3.0,
  'psa': 115.42,
  'bpka': 2.16,
  'apka': 8.62},
 {'index': 17949,
  'company': 'AAIPHARMA LLC',
  'disease_class': 'C04 Neoplasms',
  'compound': 'AZATHIOPRINE',
  'mw': 277.27,
  'clogp': 1.17,
  'a

In [43]:
with open('./DATA/cdc_descriptors_diseases.json') as data_file:
        
    descriptors_diseases_as_records=json.load(data_file)

In [44]:
descriptors_diseases_as_records['data']

[{'index': 17964,
  'company': 'AAIPHARMA LLC',
  'disease_class': 'C16 Congenital, Hereditary, and Neonatal Diseases and Abnormalities',
  'compound': 'AZATHIOPRINE',
  'mw': 277.27,
  'clogp': 1.17,
  'arom': 3.0,
  'hba': 8.0,
  'hbd': 1.0,
  'rtb': 3.0,
  'psa': 115.42,
  'bpka': 2.16,
  'apka': 8.62},
 {'index': 17979,
  'company': 'AAIPHARMA LLC',
  'disease_class': 'C15 Hemic and Lymphatic Diseases',
  'compound': 'AZATHIOPRINE',
  'mw': 277.27,
  'clogp': 1.17,
  'arom': 3.0,
  'hba': 8.0,
  'hbd': 1.0,
  'rtb': 3.0,
  'psa': 115.42,
  'bpka': 2.16,
  'apka': 8.62},
 {'index': 17974,
  'company': 'AAIPHARMA LLC',
  'disease_class': 'C10 Nervous System Diseases',
  'compound': 'AZATHIOPRINE',
  'mw': 277.27,
  'clogp': 1.17,
  'arom': 3.0,
  'hba': 8.0,
  'hbd': 1.0,
  'rtb': 3.0,
  'psa': 115.42,
  'bpka': 2.16,
  'apka': 8.62},
 {'index': 17949,
  'company': 'AAIPHARMA LLC',
  'disease_class': 'C04 Neoplasms',
  'compound': 'AZATHIOPRINE',
  'mw': 277.27,
  'clogp': 1.17,
  'a

In [45]:

normalized_df1 = json_normalize(descriptors_companies_as_records['data'])
normalized_df1.to_csv('./DATA/cdc_descriptors_companies.csv',index=False)

In [46]:

normalized_df2 = json_normalize(descriptors_diseases_as_records['data'])
normalized_df2.to_csv('./DATA/cdc_descriptors_diseases.csv',index=False)